## Exploration of map reduce result

---

### Import Libraries

In [1]:
# import libraries
import findspark
import pandas as pd
import pyspark as ps
from pyspark.sql.functions import col, sum
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType
from pyspark.sql import SparkSession


### Initialize Spark

In [10]:
# Locate the spark installation
findspark.init()

# Initialize a SparkContext
spark = SparkSession.builder.appName("data_cleaning").getOrCreate()
spark.stop()
sc = ps.SparkContext(appName="prior_analysis")
# Initialize the Session
spark_session = ps.sql.SparkSession(sc)

23/09/08 14:36:43 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


### Connect and import data from HDFS directly into a Spark DataFrame

In [26]:
# Title, description, authors, publisher, publishedDate, categories, Price, User_id, profileName, review/helpfulness, review/score, review/time, review/summary, review/text

joined_schema = StructType([
    StructField("Title", StringType(), True),
    StructField("description", StringType(), True),
    StructField("authors", StringType(), True),
    StructField("publisher", StringType(), True),
    StructField("publishedDate", StringType(), True),
    StructField("categories", StringType(), True),
    StructField("Price", FloatType(), True),
    StructField("User_id", IntegerType(), True),
    StructField("profileName", StringType(), True),
    StructField("review/helpfulness", StringType(), True),
    StructField("review/score", FloatType(), True),
    StructField("review/time", IntegerType(), True),
    StructField("review/summary", StringType(), True),
    StructField("review/text", StringType(), True)
    ])

df_join_result = spark_session.read.csv('hdfs://localhost:9900/user/book_reviews/joined_tables_v2.csv', schema=joined_schema, sep='\t')
#df_join_result.printSchema()
#df_join_result.describe().show()

df_join_result.count()

2458742

In [27]:
tmp = df_join_result.limit(300).toPandas()
tmp.head(300)

# Title, description, authors, publisher, publishedDate, categories, Price, User_id, profileName, review/helpfulness, review/score, review/time, review/summary, review/text


,Title,description,authors,publisher,publishedDate,categories,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text
0,01442 DEVELOPING SKILLS IN ALGEBRA ONE BOOK B,None,None,None,1977,American literature,NaN,NaN,M Garrison,0/0,5.0,963619200,The best source for Algebra I worksheets,None
1,01442 DEVELOPING SKILLS IN ALGEBRA ONE BOOK B,None,None,None,1977,American literature,NaN,NaN,jen,0/0,5.0,1295049600,Great book,None
2,01443 DEVELOPING SKILLS IN ALGEBRA ONE BOOK C,None,Rose Arny,None,1995,American literature,23.709999,NaN,None,4/4,5.0,963619200,The best source for Algebra I worksheets,None
3,01443 DEVELOPING SKILLS IN ALGEBRA ONE BOOK C,None,Rose Arny,None,1995,American literature,23.709999,NaN,Magdalena Vazquez,0/0,5.0,1325030400,Best practice workbook in my 25 years of teaching,None
4,01443 DEVELOPING SKILLS IN ALGEBRA ONE BOOK C,None,Rose Arny,None,1995,American literature,23.709999,NaN,C Sefchik book worm,0/0,5.0,1319587200,Algebra problems Book C,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,1 000 Places to See Before You Die A Traveler...,The world’s bestselling travel book is back in...,Patricia Schultz,Workman Publishing,2015-07-01,Travel,NaN,NaN,Eileen Rieback,19/21,4.0,1082592000,Wonders of the world,None
296,1 000 Places to See Before You Die A Traveler...,The world’s bestselling travel book is back in...,Patricia Schultz,Workman Publishing,2015-07-01,Travel,NaN,NaN,None,33/39,3.0,1069286400,Fluffy but Fine,None
297,1 000 Places to See Before You Die A Traveler...,The world’s bestselling travel book is back in...,Patricia Schultz,Workman Publishing,2015-07-01,Travel,NaN,NaN,Michael W Dobson mapman,15/16,3.0,1101427200,Not for the rest of us,None
298,1 000 Places to See Before You Die A Traveler...,The world’s bestselling travel book is back in...,Patricia Schultz,Workman Publishing,2015-07-01,Travel,NaN,NaN,Frank Turner Seasoned Traveler,11/11,2.0,1134604800,Hotels to See Before You Die,None


In [5]:
spark_session.stop()